# FINM32000 HW3

## P1.

(a)

$dC_t = \frac{\partial C}{\partial r_t}dr_t+\frac{\partial C}{\partial t}dt+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}(dr_t)^2$

$= \alpha(r_t,t)\frac{\partial C}{\partial r_t}dt+\beta(r_t,t)\frac{\partial C}{\partial r_t}dW_t+\frac{\partial C}{\partial t}dt+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}\beta^2(r_t,t)dt$

$=(\alpha(r_t,t)\frac{\partial C}{\partial r_t}+\frac{1}{2}\frac{\partial^2 C}{\partial r_t^2}\beta^2(r_t,t)+\frac{\partial C}{\partial t})dt+\beta(r_t,t)\frac{\partial C}{\partial r_t}dW_t$

By setting drift equal to $rC$, we have the PDE:

$$
\alpha(r,t)\frac{\partial C}{\partial r}+\frac{1}{2}\frac{\partial^2 C}{\partial r^2}\beta^2(r,t)+\frac{\partial C}{\partial t}=rC(r,t)
$$

$$C(r_T,T)=F(r_T)$$

(b)

$\kappa(\theta-r_j)\frac{\partial C}{\partial r}+\frac{1}{2}\frac{\partial^2 C}{\partial r^2}\sigma^2+\frac{\partial C}{\partial t}=r_jC(r_j,t)$

$\frac{\partial C}{\partial r}\approx\frac{C^{j+1}_{n+1}-C^{j-1}_{n+1}}{2\Delta r}$

$\frac{\partial^2 C}{\partial r^2}\approx\frac{C^{j+1}_{n+1}-2C^{j}_{n+1}+C^{j-1}_{n+1}}{(\Delta r)^2}$

$\frac{\partial C}{\partial t}\approx\frac{C^{j}_{n+1}-C^{j}_{n}}{\Delta t}$

$\kappa(\theta-r_j)\frac{C^{j+1}_{n+1}-C^{j-1}_{n+1}}{2\Delta r}+\frac{1}{2}\sigma^2\frac{C^{j+1}_{n+1}-2C^{j}_{n+1}+C^{j-1}_{n+1}}{(\Delta r)^2}+\frac{C^{j}_{n+1}-C^{j}_{n}}{\Delta t}=r_jC_{n}^{j}$

$C_n^j=\frac{1}{1+r_j\Delta t}(q_uC^{j+1}_{n+1}+q_mC^{j}_{n+1}+q_dC^{j-1}_{n+1})$

where

$q_u = \frac{1}{2}\big[\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}+\frac{\sigma^2\Delta t}{(\Delta r)^2}\big]$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}$

$q_u = \frac{1}{2}\big[-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}+\frac{\sigma^2\Delta t}{(\Delta r)^2}\big]$

In [1]:
import numpy as np

In [2]:
class Dynamics:
    pass

In [3]:
hw3dynamics=Dynamics()
hw3dynamics.kappa = 3
hw3dynamics.theta = 0.05
hw3dynamics.sigma = 0.03

In [4]:
class Contract:
    pass

In [5]:
hw3contract=Contract()
hw3contract.T = 5

In [6]:
class FD:
    pass

In [7]:
hw3FD=FD()
hw3FD.rMax=0.35
hw3FD.rMin=-0.25
hw3FD.deltar=0.01
hw3FD.deltat=0.01
hw3FD.useUpwind=False


In [10]:
# You complete the coding of this function

def pricer_bond_vasicek_explicitFD(contract,dynamics,fd):
# returns array of all initial short rates,
# and the corresponding array of zero-coupon
# T-maturity bond prices

    T = contract.T
    kappa, theta, sigma = dynamics.kappa, dynamics.theta, dynamics.sigma    
    rMax, rMin, deltar, deltat = fd.rMax, fd.rMin, fd.deltar, fd.deltat    
    N=round(T/deltat)
    if abs(N-T/deltat) > 1e-12:
        raise ValueError("Bad delta t")
        
    r=np.arange(rMax,rMin-deltar/2,-deltar)   #I'm making the FIRST indices of the array correspond to HIGH levels of r
    bondprice=np.ones(np.size(r))
    a=kappa*(theta-r)*deltat/deltar
    b=(sigma**2*deltat/(deltar**2))
  
    if fd.useUpwind:
        qu=    0.5*(b+a+abs(a))
        qd=    0.5*(b-a+abs(a))
        qm=    1-qu-qd
        
    else: ##central difference
        qu=    0.5*(a+b)
        qm=    1-b
        qd=    0.5*(b-a)
        
    for t in np.arange(N-1,-1,-1)*deltat:
        bondprice=1/(1+r*deltat)*(qd*np.roll(bondprice,-1)+qm*bondprice+qu*np.roll(bondprice,1))
        
        # It is not obvious in this case, 
        # what boundary conditions to impose at the top and bottom
        # so let us impose "linearity" boundary conditions
        bondprice[0]=2*bondprice[1]-bondprice[2]
        bondprice[-1]=2*bondprice[-2]-bondprice[-3]
        
    return (r, bondprice)

In [11]:
(r, bondprice) = pricer_bond_vasicek_explicitFD(hw3contract,hw3dynamics,hw3FD)

In [12]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
displayrows=np.logical_and(r<0.15+hw3FD.deltar/2, r>0.0-hw3FD.deltar/2)
answer_central=np.stack((r, bondprice),1)[displayrows]

In [13]:
print(answer_central)

[[ 1.5000e-01 -1.4273e+09]
 [ 1.4000e-01  1.6361e+08]
 [ 1.3000e-01  2.2294e+07]
 [ 1.2000e-01 -1.3724e+06]
 [ 1.1000e-01 -1.3361e+05]
 [ 1.0000e-01  3.2966e+03]
 [ 9.0000e-02  1.3021e+02]
 [ 8.0000e-02  7.7128e-01]
 [ 7.0000e-02  7.7385e-01]
 [ 6.0000e-02  7.7643e-01]
 [ 5.0000e-02  7.7902e-01]
 [ 4.0000e-02  7.8162e-01]
 [ 3.0000e-02  7.8423e-01]
 [ 2.0000e-02  7.8685e-01]
 [ 1.0000e-02  1.4165e+03]
 [-3.3307e-16  5.1498e+04]]


(c)

When $\kappa(\theta-r_j)\ge 0$

$\frac{\partial C}{\partial r}\approx\frac{C^{j+1}_{n+1}-C^{j}_{n+1}}{\Delta r}$

$\kappa(\theta-r_j)\frac{C^{j+1}_{n+1}-C^{j}_{n+1}}{\Delta r}+\frac{1}{2}\sigma^2\frac{C^{j+1}_{n+1}-2C^{j}_{n+1}+C^{j-1}_{n+1}}{(\Delta r)^2}+\frac{C^{j}_{n+1}-C^{j}_{n}}{\Delta t}=r_jC_{n}^{j}$

$q_u = \frac{\kappa(\theta-r_j)\Delta t}{\Delta r}+\frac{\sigma^2\Delta t}{2(\Delta r)^2}$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}$

$q_d = \frac{\sigma^2\Delta t}{2(\Delta r)^2}$

When $\kappa(\theta-r_j) < 0$

$\frac{\partial C}{\partial r}\approx\frac{C^{j+1}_{n}-C^{j}_{n-1}}{\Delta r}$

$\kappa(\theta-r_j)\frac{C^{j}_{n+1}-C^{j-1}_{n+1}}{\Delta r}+\frac{1}{2}\sigma^2\frac{C^{j+1}_{n+1}-2C^{j}_{n+1}+C^{j-1}_{n+1}}{(\Delta r)^2}+\frac{C^{j}_{n+1}-C^{j}_{n}}{\Delta t}=r_jC_{n}^{j}$

$q_u = \frac{\sigma^2\Delta t}{2(\Delta r)^2}$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}+\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}$

$q_d = \frac{\sigma^2\Delta t}{2(\Delta r)^2}-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}$

Or to sum up for easier implementation:

$q_u = \frac{\sigma^2\Delta t}{2(\Delta r)^2}+(\frac{\kappa(\theta-r_j)\Delta t}{\Delta r})^+$

$q_m = 1-\frac{\sigma^2\Delta t}{(\Delta r)^2}-\big|\frac{\kappa(\theta-r_j)\Delta t}{\Delta r}\big|=1-q_u-q_d$

$q_u = \frac{\sigma^2\Delta t}{2(\Delta r)^2}+(-\frac{\kappa(\theta-r_j)\Delta t}{\Delta r})^+$

In [14]:
hw3FD.useUpwind=True
(r, bondprice) = pricer_bond_vasicek_explicitFD(hw3contract,hw3dynamics,hw3FD)

In [15]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
displayrows=np.logical_and(r<0.15+hw3FD.deltar/2, r>0.0-hw3FD.deltar/2)
answer_upwind = np.stack((r, bondprice),1)[displayrows]

In [16]:
print(answer_upwind)

[[ 0.15    0.7536]
 [ 0.14    0.7561]
 [ 0.13    0.7586]
 [ 0.12    0.7611]
 [ 0.11    0.7637]
 [ 0.1     0.7662]
 [ 0.09    0.7688]
 [ 0.08    0.7713]
 [ 0.07    0.7739]
 [ 0.06    0.7765]
 [ 0.05    0.7791]
 [ 0.04    0.7817]
 [ 0.03    0.7843]
 [ 0.02    0.7869]
 [ 0.01    0.7895]
 [-0.      0.7922]]


(d)

By Taylor's theorem:

$f(x+h)=f(x)+hf'(x)+\frac{1}{2}h^2f''(x)+O(h^3)$

$f(x-h)=f(x)-hf'(x)+\frac{1}{2}h^2f''(x)+O(h^3)$

$\therefore\big|\frac{f(x+h)-f(x)}{h}-f'(x)\big|=\frac{1}{2}hf''(x)+\frac{O(h^3)}{h}=O(h)$

$\big|\frac{f(x+h)-f(x-h)}{2h}-f'(x)\big|=\frac{O(h^3)}{h}=O(h^2)$

(e)

In [17]:
answer_central[abs(answer_central[:,0]-0.1)<1e-12][0,1]

3296.5929237489718

In [18]:
answer_upwind[abs(answer_upwind[:,0]-0.1)<1e-12][0,1]

0.7662252882450523

The upwind calculation is much more accurate

(f)less;greater

(g)

In [17]:
#0.12 case
pt = answer_upwind[abs(answer_central[:,0]-0.12)<1e-12][0,1]  
ytm = np.log(abs(1/pt))/5.0
ytm

0.05458660238621129

In [18]:
#0.02 case
pt = answer_upwind[abs(answer_central[:,0]-0.02)<1e-12][0,1]
ytm = np.log(abs(1/pt))/5.0
ytm

0.047926845240380489

Intuitions:
Intuitively, the mean of the interest rate is 0.05 as $\theta = 0.05$ 

So given $r_0 = 0.12 > 0.05 = \theta$, we may expect $r_0>ytm>\theta$. 

Similarly, when $r_0 = 0.02$, $r_0 < ytm < \theta$

## P2

(a) By lecture note:

$\sigma_{imp}=\bar{\sigma_t}=\sqrt{\frac{1}{T}\int_0^T\sigma^2(t)dt}$

which is a function of $T$ but not $K$

Therefore the dynamics can generate non-constant term-structure of implied volatility but not an implied volatility skew.

(b)

Borrow the IVofCall function from HW1:

In [19]:
from scipy.stats import norm
from scipy.optimize import bisect

def BScallPrice(sigma,S,rGrow,r,K,T):
    F=S*np.exp(rGrow*T)
    sd = sigma*np.sqrt(T)
    d1 = np.log(F/K)/sd+sd/2
    d2 = d1-sd
    return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))

def IVofCall(C,S,rGrow,r,K,T):
    F=S*np.exp(rGrow*T)
    lowerbound = np.max([0,(F-K)*np.exp(-r*T)])
    if C<lowerbound:
        return np.nan
    if C==lowerbound:
        return 0
    if C>=F*np.exp(-r*T):
        return np.nan 
    hi=0.2
    while BScallPrice(hi,S,rGrow,r,K,T)>C:
        hi=hi/2
    while BScallPrice(hi,S,rGrow,r,K,T)<C:
        hi=hi*2
    lo=hi/2
    impliedVolatility = bisect(lambda x: BScallPrice(x,S,rGrow,r,K,T)-C,hi,lo)   # you fill this in, using bisect or brentq imported from scipy.optimize,
    return impliedVolatility

In [20]:
S = 100
K = 100
rGrow = r = 0.05
T = [0.1,0.2,0.5]
C = [5.25,7.25,9.5]
imp_vols = []
for i in range(3):
    imp_vols.append(IVofCall(C[i],S,rGrow,r,K,T[i]))
print(imp_vols)

[0.3973203857967747, 0.3801712915519602, 0.2950972521750374]


Denote the 3 answers as $\sigma_1,\sigma_2,\sigma_3$

We want to create a step function such that 

$\bar{\sigma_t}=\sqrt{\frac{1}{T}\int_0^T\sigma^2(t)dt}$

has value $\sigma_1,\sigma_2,\sigma_3$ at $t = 0.1,0.2,0.5$

A local volatility function would be:

$\sigma(t)=\begin{cases} 
      \sigma_1 & 0 \leq t\leq 0.1 \\
      \sqrt{2\sigma_2^2-\sigma_1^2} & 0.1 < t \leq 0.2\\
      \sqrt{\frac{1}{3}(5\sigma_3^2-2\sigma_2^2)} & 0.2 < t \leq 0.5
\end{cases}$



(c)

$\bar\sigma_{0.4}=\sqrt{\frac{1}{0.4}\int_0^{0.4}\sigma^2(t)dt}$

$=\sqrt{\frac{1}{0.4}*(0.1\sigma_1^2+0.1(2\sigma_2^2-\sigma_1^2)+0.2(\frac{1}{3}(5\sigma_3^2-2\sigma_2^2))}$

$=\sqrt{\frac{5}{6}\sigma_3^2+\frac{1}{6}\sigma^2_2}$

The exact value as well as the final price calculated in code below

In [21]:
vol_c = ((5*imp_vols[2]**2/6)+(1*imp_vols[1]**2/6))** 0.5
T_c = 0.4
price = BScallPrice(vol_c,S,rGrow,r,K,T_c)
print("price of European call with expiry 0.4: %s" %price)

price of European call with expiry 0.4: 8.78420177592
